In [1]:
import concurrent
from moviepy.editor import VideoFileClip
from lesson_functions import *
import matplotlib.pyplot as plt
import matplotlib
from scipy.ndimage.measurements import label
import pickle
from collections import OrderedDict
from os.path import join

# Setup

## Parameters

In [2]:
color_space = 'HLS'  # Can be RGB, HSV, LUV, HLS, YUV, YCrCb
orient = 9  # HOG orientations
pix_per_cell = 16  # HOG pixels per cell
cell_per_block = 2  # HOG cells per block
hog_channel = "ALL"  # Can be 0, 1, 2, or "ALL"
spatial_size = (16, 16)  # Spatial binning dimensions
hist_bins = 16  # Number of histogram bins
spatial_feat = True  # Spatial features on or off
hist_feat = True  # Histogram features on or off
hog_feat = True  # HOG features on or off

In [3]:
window_params = OrderedDict([
    (64, {"xy_overlap": (0.75, 0.75), "x_start_stop": (120, 1280-120), "y_start_stop": (375, 500)}),
    (70, {"xy_overlap": (0.75, 0.75), "x_start_stop": (60, 1280-60), "y_start_stop": (375, 500)}),
    (90, {"xy_overlap": (0.75, 0.75), "x_start_stop": (0, 1280), "y_start_stop": (375, 560)}),
    (115, {"xy_overlap": (0.5, 0.5), "x_start_stop": (0, 1280), "y_start_stop": (375, 600)}),
    (154, {"xy_overlap": (0.5, 0.5), "x_start_stop": (0, 1280), "y_start_stop": (400, 680)}),
    (185, {"xy_overlap": (0.5, 0.5), "x_start_stop": (0, 1280), "y_start_stop": (450, 680)}),
    (218, {"xy_overlap": (0.5, 0.5), "x_start_stop": (0, 1280), "y_start_stop": (450, 680)}),
])

## Unpickle classifier and scaler

In [5]:
file_name = "svc.p" #"0.0.6_svc.p"
with open(file_name, "rb") as ifile:
    svc, X_scaler = pickle.load(ifile)

# Functions and class

In [6]:
def run_windowing_and_detection_with_image(arg):
    xy_size, param_dict, image = arg[0], arg[1], arg[2]
    windows = slide_window_from_bottom(
        image, x_start_stop=param_dict["x_start_stop"], y_start_stop=param_dict["y_start_stop"],
        xy_window=(xy_size, xy_size), xy_overlap=param_dict["xy_overlap"])

    hot_windows = search_windows(image, windows, svc, X_scaler, color_space=color_space,
                                 spatial_size=spatial_size, hist_bins=hist_bins,
                                 orient=orient, pix_per_cell=pix_per_cell,
                                 cell_per_block=cell_per_block,
                                 hog_channel=hog_channel, spatial_feat=spatial_feat,
                                 hist_feat=hist_feat, hog_feat=hog_feat)
    return hot_windows


def add_heat(heatmap, bbox_list):
    # Iterate through list of bboxes
    for box in bbox_list:
        # Add += 1 for all pixels inside each bbox
        # Assuming each "box" takes the form ((x1, y1), (x2, y2))
        heatmap[box[0][1]:box[1][1], box[0][0]:box[1][0]] += 1

    # Return updated heatmap
    return heatmap  # Iterate through list of bboxes


def apply_threshold(heatmap, threshold):
    # Zero out pixels below the threshold
    heatmap[heatmap <= threshold] = 0
    # Return thresholded map
    return heatmap


def draw_labeled_bboxes(img, labels):
    # Iterate through all detected cars
    for car_number in range(1, labels[1] + 1):
        # Find pixels with each car_number label value
        nonzero = (labels[0] == car_number).nonzero()
        # Identify x and y values of those pixels
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])
        # Define a bounding box based on min/max x and y
        bbox = ((np.min(nonzerox), np.min(nonzeroy)), (np.max(nonzerox), np.max(nonzeroy)))
        # Draw the box on the image
        cv2.rectangle(img, bbox[0], bbox[1], (0, 0, 255), 6)
    # Return the image
    return img

In [7]:
class VehicleDetection:
    def __init__(self, n_smoothing_frames=10, threshold_multiplier=8):
        self.boxes_queue = []
        self.n_smoothing_frames = n_smoothing_frames
        self.base_heat = None
        self.threshold_multiplier = threshold_multiplier

    def add_boxes_set(self, boxes):
        self.boxes_queue.append(boxes)
        if len(self.boxes_queue) > self.n_smoothing_frames:
            self.boxes_queue = self.boxes_queue[-self.n_smoothing_frames:]

    def get_all_queue_boxes(self):
        return [box for sublist in self.boxes_queue for box in sublist]

    def get_base_heatmap(self, img):
        if self.base_heat is None:
            self.base_heat = np.zeros_like(img[:, :, 0]).astype(np.float)
        return self.base_heat.copy()

    def get_threshold(self):
        # threshold = len(self.boxes_queue)*self.threshold_multiplier
        # threshold = len(self.get_all_queue_boxes())*self.threshold_multiplier
        threshold = self.threshold_multiplier
        return threshold

    def _get_boxes(self, cnv):
        boxes = []
        with concurrent.futures.ProcessPoolExecutor() as executor:
            for new_boxes in executor.map(run_windowing_and_detection_with_image,
                                          [(k, v, cnv) for k, v in window_params.items()]):
                boxes += new_boxes
        return boxes

    def run(self, img):
        # detect all boxes
        cnv = convert_to_float_if_required(img)
        boxes = self._get_boxes(cnv)

        # add to queue, get all the ones for smoothing
        self.add_boxes_set(boxes)
        boxes4smoothing = self.get_all_queue_boxes()

        # create a heat map for filtering false positives
        heat = add_heat(self.get_base_heatmap(img), boxes4smoothing)
        heat = apply_threshold(heat, self.get_threshold())
        heatmap = np.clip(heat, 0, 255)
        # Find final boxes from heatmap using label function
        labels = label(heatmap)
        img = draw_labeled_bboxes(img, labels)

        # img = draw_boxes(img, boxes, color=(0, 0, 255), thick=4)
        return img

    def run_debug(self, img):
        # detect all boxes
        cnv = convert_to_float_if_required(img)
        boxes = self._get_boxes(cnv)

        # add to queue, get all the ones for smoothing
        self.add_boxes_set(boxes)
        boxes4smoothing = self.get_all_queue_boxes()

        # create a heat map for filtering false positives
        heat = add_heat(self.get_base_heatmap(img), boxes4smoothing)
        heat = apply_threshold(heat, self.get_threshold())
        heatmap = np.clip(heat, 0, 255)
        # Find final boxes from heatmap using label function
        labels = label(heatmap)

        # plotting
        grid_shape = (2, 2)
        figure = plt.figure(figsize=[16, 10])
        to_plot = [
            ((0, 0), draw_labeled_bboxes(img.copy(), labels), "Project output", {}),
            ((0, 1), draw_boxes(img.copy(), boxes, color=(0, 0, 255), thick=4), "Boxes detected this frame", {}),
            ((1, 1), draw_boxes(img.copy(), boxes4smoothing, color=(0, 0, 255), thick=4),
             "Boxes detected last {} frames".format(self.n_smoothing_frames), {}),
            ((1, 0), heatmap, "Heatmap of boxes detect in last {} frames".format(self.n_smoothing_frames),
             {"cmap": "hot"})
        ]

        for pos, data, title, kwargs in to_plot:
            ax = plt.subplot2grid(grid_shape, pos)
            ax.imshow(data, **kwargs)
            ax.set_title(title)
            ax.set_xticks([])
            ax.set_yticks([])

        plt.tight_layout()
        figimg = self.convert_to_image_smart()
        plt.close(figure)
        return figimg

    @staticmethod
    def convert_to_image_smart():
        figure = plt.gcf()

        # remove anti aliasing, might not be necessary
        matplotlib.rcParams['text.antialiased'] = False
        for ax in figure.axes:
            plt.setp(
                [ax.get_xticklines() + ax.get_yticklines() + ax.get_xgridlines() + ax.get_ygridlines()],
                antialiased=False)

        # draw the figure
        figure.canvas.draw()

        # Now we can save it to a numpy array.
        data = np.fromstring(figure.canvas.tostring_rgb(), dtype=np.uint8, sep='')
        return data.reshape(figure.canvas.get_width_height()[::-1] + (3,))

# Running

In [9]:
video_name = "project_video.mp4"
output_directory = "."

n_frames = 50
thresh = 230
vd = VehicleDetection(n_smoothing_frames=n_frames, threshold_multiplier=thresh)
out_fn = "0.0.6_frames_{}_thresh_{}_{}".format(n_frames, thresh, video_name)

clip = VideoFileClip("./" + video_name)
project_clip = clip.fl_image(vd.run)
project_clip.write_videofile(join(output_directory, out_fn), audio=False)
print("Output video: {}".format(out_fn))



[MoviePy] >>>> Building video ./0.0.6_frames_50_thresh_230_project_video.mp4
[MoviePy] Writing video ./0.0.6_frames_50_thresh_230_project_video.mp4


100%|█████████▉| 1260/1261 [42:09<00:02,  2.05s/it]


[MoviePy] Done.
[MoviePy] >>>> Video ready: ./0.0.6_frames_50_thresh_230_project_video.mp4 

Output video: 0.0.6_frames_50_thresh_230_project_video.mp4
